<a href="https://colab.research.google.com/github/lekejo/lekejo/blob/main/GenBriefs_No_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# This is a Google Colab Python Notebook. If you have a Google(Gmail) account, go to https://colab.research.google.com/, open this notebook and then run (play button) cell by cell (the first run will initiate a server automatically, wait for the play button to become a number - seconds and then run the next cell, etc). Easyyy!
# Comment the following if being run for the second time - This will install necessary packages for working with MS Word document.
!pip install python-docx


In [42]:
#DANGER THE FOLLLOWING LINE WILL CLEAR BRIEFS FOLDER - KEEP THIS COMMENTED IF YOU WANT TO HOLD ON TO EXISTING DOCUMENTS
!rm -rf briefs

In [43]:
# Copyright Central Informatics Bureau - the one behind the original GenSpecs - You know who.
# Program Description: Asks user to upload an excel containing projects in the format of current CIB Project dashboard and then program will generate project briefs for New and Ongoing projects only
import pandas as pd
from google.colab import files
from docx import Document
from datetime import datetime
import logging
import time
import os
import re
import unicodedata
from tqdm.auto import tqdm

# --- Parameterized values ---
OUTPUT_FOLDER = "briefs"  # Folder to store the generated briefs
LOG_FOLDER = "logs"  # Folder to store the log file
ALLOWED_TYPES = ('ongoing', 'new')  # Allowed values for the 'Type' column
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'  # Expected date format in the Excel file
TYPE_COLUMN = 'Type'  # Column name for project type
PROJECT_NAME_COLUMN = 'Project Name'  # Column name for project name
MINISTRY_DEPT_COLUMN = 'Ministry/Dept'  # Column name for ministry/department
DESCRIPTION_COLUMN = 'Description'  # Column name for project description
CONTRACT_VALUE_COLUMN = 'Contract Value'  # Column name for contract value
START_DATE_COLUMN = 'Start Date'  # Column name for project start date
COMPLETION_DATE_COLUMN = 'Completion Date'  # Column name for project completion date
STATUS_COLUMN = 'Status'  # Column name for project status
PID_COLUMN = 'PID'  # Column name for project ID

# Columns to check for issues
COLUMNS_TO_CHECK = [
    TYPE_COLUMN,
    PROJECT_NAME_COLUMN,
    MINISTRY_DEPT_COLUMN,
    DESCRIPTION_COLUMN,
    CONTRACT_VALUE_COLUMN,
    START_DATE_COLUMN,
    COMPLETION_DATE_COLUMN,
    STATUS_COLUMN,
    PID_COLUMN
]

# Flag to enable/disable summary check
PERFORM_SUMMARY_CHECK = True

# --- Function definitions ---

def generate_project_brief(row):
    """Generates a project brief for a single project as a Word document."""
    try:
        project_name = str(row[PROJECT_NAME_COLUMN])  # Get project name
        project_type = row[TYPE_COLUMN].capitalize()  # Get and capitalize project type
        try:
            pid = str(int(row[PID_COLUMN])).zfill(3)  # Get and format PID with leading zeros
        except ValueError:
            pid = "XXX"  # Default PID if not an integer

        logging.debug(f"Generating brief for: {project_name} (PID: {pid})")  # Log project details

        if project_type.lower() not in ALLOWED_TYPES:  # Check if project type is allowed
            logging.debug(f"Skipping {project_name}, status: {project_type}")  # Log skipped project
            return False

        document = Document()  # Create a new Word document
        document.add_heading(project_name, level=1)  # Add project name as heading

        # Add sections to the document
        add_brief_section(document, "Ministry/Department:", row[MINISTRY_DEPT_COLUMN])
        add_brief_section(document, "Project Description", row[DESCRIPTION_COLUMN])
        add_brief_section(document, "Project Value:", row[CONTRACT_VALUE_COLUMN])

        # Format dates
        try:
            start_date = datetime.strptime(str(row[START_DATE_COLUMN]), DATE_FORMAT).strftime('%d %B %Y')
        except ValueError:
            start_date = "N/A"
        try:
            completion_date = datetime.strptime(str(row[COMPLETION_DATE_COLUMN]), DATE_FORMAT).strftime('%d %B %Y')
        except ValueError:
            completion_date = "N/A"
        add_brief_section(document, "Timeframe:", f"{start_date} to {completion_date}")
        add_brief_section(document, "Status:", row[STATUS_COLUMN])

        file_name = sanitize_filename(f"{project_type}_{pid}_{project_name}_Brief.docx")  # Create file name
        file_path = os.path.join(OUTPUT_FOLDER, file_name)  # Create file path
        document.save(file_path)  # Save the document
        logging.debug(f"Brief saved for: {project_name} as {file_name}")  # Log saved brief
        return True
    except Exception as e:
        logging.error(f"Error generating brief for {row[PROJECT_NAME_COLUMN]}: {e}", exc_info=True)  # Log error
        return False

def add_brief_section(document, header, content):
    """Adds a section with header and content to the document."""
    document.add_paragraph(header, style='Heading 2')  # Add header
    if isinstance(content, (int, float)):
        content = f"{content:,}"  # Format numbers with commas
    document.add_paragraph(str(content))  # Add content

def sanitize_filename(filename):
    """Removes invalid characters and normalizes filename."""
    filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode('ASCII')  # Normalize
    filename = re.sub(r'[\\/:*?"<>|]', "-", filename)  # Replace invalid characters
    return filename.strip().strip('.')  # Remove leading/trailing spaces and periods

def check_for_issues(df):
    """Checks for issues in the DataFrame and returns a summary."""
    issues = {}
    for index, row in df.iterrows():
        project_name = str(row[PROJECT_NAME_COLUMN])
        try:  # Robust handling of PID conversion
            pid = str(int(row[PID_COLUMN])).zfill(3)
        except ValueError:
            logging.error(f"Invalid PID format for project: {project_name}. Skipping.")
            continue

        project_id = f"{pid}-{project_name}"
        issues[project_id] = {}
        for column in COLUMNS_TO_CHECK:
            issues[project_id][column] = []
            try:  # Robust handling of data types and potential errors
                if pd.isnull(row[column]):
                    issues[project_id][column].append("Empty cell")
                if column == CONTRACT_VALUE_COLUMN and not isinstance(row[column], (int, float)):
                    issues[project_id][column].append("Non-numeric value")
                if column in (START_DATE_COLUMN, COMPLETION_DATE_COLUMN):
                    datetime.strptime(str(row[column]), DATE_FORMAT)  # Check date format
            except (ValueError, TypeError) as e:
                issues[project_id][column].append("Invalid value")
                logging.error(f"Error checking column {column} for {project_id}: {e}, Value: {row[column]}")
    return issues

def main():
    """Main function to execute the script."""
    if not os.path.exists(OUTPUT_FOLDER):  # Create output folder if it doesn't exist
        os.makedirs(OUTPUT_FOLDER)
    if not os.path.exists(LOG_FOLDER):  # Create logs folder if it doesn't exist
        os.makedirs(LOG_FOLDER)

    timestamp = time.strftime("%Y%m%d-%H%M%S")  # Generate timestamp for log file name
    log_file_name = os.path.join(LOG_FOLDER, f"brief_gen_{timestamp}.log")  # Create log file name

    # Configure logging
    logging.basicConfig(filename=log_file_name, level=logging.DEBUG,
                        force=True,  # Force creation of log file
                        format='%(asctime)s - %(levelname)s - %(message)s')

    uploaded = files.upload()  # Upload Excel file
    filename = next(iter(uploaded))  # Get filename

    try:
        df = pd.read_excel(filename)  # Read Excel file into a DataFrame
        logging.debug("Excel file loaded successfully.")  # Log successful file load
    except Exception as e:
        logging.error(f"Error loading Excel file: {e}", exc_info=True)  # Log error
        raise e

    total_projects = len(df)  # Get total number of projects
    new_projects = len(df[df[TYPE_COLUMN].str.lower() == 'new'])  # Get number of new projects
    ongoing_projects = len(df[df[TYPE_COLUMN].str.lower() == 'ongoing'])  # Get number of ongoing projects

    print(f"Found {total_projects} projects in the spreadsheet.")  # Print total projects
    logging.debug(f"Total projects: {total_projects}")  # Log total projects
    logging.debug(f"New projects: {new_projects}")  # Log new projects
    logging.debug(f"Ongoing projects: {ongoing_projects}")  # Log ongoing projects

    briefs_created = 0  # Initialize counter for created briefs
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Creating Briefs"):  # Iterate over projects and generate briefs
        if generate_project_brief(row):
            briefs_created += 1

    logging.debug("Brief generation completed.")  # Log completion of brief generation
    logging.debug(f"Briefs created: {briefs_created}")  # Log number of briefs created

    print(f"\nOut of {total_projects} projects, {new_projects} were New and {ongoing_projects} were Ongoing.")  # Print summary of project types
    print(f"Total briefs created: {briefs_created}")  # Print total briefs created
    print(f"Log file created at: {log_file_name}")  # Print log file location

    if PERFORM_SUMMARY_CHECK:
        df_filtered = df[df[TYPE_COLUMN].str.lower().isin(ALLOWED_TYPES)]
        issues = check_for_issues(df_filtered)

        print("\n--- Issue Summary ---")
        if any(issues.values()):
            issue_summary = {}
            for project_id, project_issues in issues.items():
                for column, issue_list in project_issues.items():
                    for issue in issue_list:
                        if column not in issue_summary:
                            issue_summary[column] = {}
                        if issue not in issue_summary[column]:
                            issue_summary[column][issue] = 0
                        issue_summary[column][issue] += 1

            # Print and log the summarized issues per column with counts
            for column, issue_counts in issue_summary.items():
                for issue, count in issue_counts.items():
                    message = f"{column}: {count} {issue} issue(s)"
                    print(message)
                    logging.warning(message)

            logging.warning("\n--- Breakdown of Issues per Project ---")
            for project_id, project_issues in issues.items():
                if any(project_issues.values()):
                    logging.warning(f"{project_id}:")
                    for column, issue_list in project_issues.items():
                        for issue in issue_list:
                            logging.warning(f"  - {column}: {issue}")
        else:  # This is the else block you requested
            print("No issues found.")
            logging.info("No issues found.")

if __name__ == "__main__":
    main()  # Execute the main function

Saving ProjectStatus.xlsx to ProjectStatus (19).xlsx
Found 291 projects in the spreadsheet.


Creating Briefs:   0%|          | 0/291 [00:00<?, ?it/s]


Out of 291 projects, 80 were New and 42 were Ongoing.
Total briefs created: 122
Log file created at: logs/brief_gen_20240921-084557.log

--- Issue Summary ---
Contract Value: 92 Non-numeric value issue(s)
Contract Value: 28 Empty cell issue(s)
Completion Date: 35 Invalid value issue(s)
Status: 2 Empty cell issue(s)
Start Date: 8 Invalid value issue(s)
